# Hibernating Tenrec ATAC-seq Analysis

This notebook analyzes ATAC-seq data from hibernating tenrecs, focusing on:
1. TSS enrichment analysis
2. Peak annotation
3. Motif analysis

**Author:** [Your Name]  
**Date:** [Date]  
**Project:** Tenrec Hibernation Study

## 1. Setup and Imports

In [ ]:
# Standard libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Bioinformatics libraries
import pysam
import pybedtools
from pybedtools import BedTool

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")

## 2. Define Paths and Parameters

In [ ]:
# Data paths
BASE_DIR = Path('/path/to/tenrec/data')
BAM_DIR = BASE_DIR / 'bam_files'
PEAK_DIR = BASE_DIR / 'peaks'
GENOME_DIR = BASE_DIR / 'genome'
OUTPUT_DIR = BASE_DIR / 'analysis_output'

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Sample information
SAMPLES = {
    'hibernating': ['hib_rep1', 'hib_rep2', 'hib_rep3'],
    'active': ['active_rep1', 'active_rep2', 'active_rep3']
}

# Genome files
GENOME_FASTA = GENOME_DIR / 'tenrec_genome.fa'
GTF_FILE = GENOME_DIR / 'tenrec_annotation.gtf'
TSS_BED = GENOME_DIR / 'tenrec_tss.bed'  # Pre-computed TSS regions

# Analysis parameters
TSS_WINDOW = 2000  # bp around TSS for enrichment calculation
PEAK_EXTENSION = 250  # bp to extend peaks for motif analysis

print(f"Output directory: {OUTPUT_DIR}")

## 3. Load ATAC-seq Data

In [ ]:
# Load peak files (MACS2 narrowPeak format)
def load_peaks(sample_name, peak_dir=PEAK_DIR):
    """
    Load ATAC-seq peaks for a given sample.
    
    Parameters:
    -----------
    sample_name : str
        Name of the sample
    peak_dir : Path
        Directory containing peak files
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with peak coordinates and statistics
    """
    peak_file = peak_dir / f"{sample_name}_peaks.narrowPeak"
    
    cols = ['chrom', 'start', 'end', 'name', 'score', 'strand', 
            'signalValue', 'pValue', 'qValue', 'peak']
    
    peaks = pd.read_csv(peak_file, sep='\t', header=None, names=cols)
    peaks['sample'] = sample_name
    
    return peaks

# Load all peaks
all_peaks = {}
for condition, samples in SAMPLES.items():
    for sample in samples:
        all_peaks[sample] = load_peaks(sample)
        print(f"Loaded {len(all_peaks[sample])} peaks for {sample}")

# Combine peaks from all samples
peaks_df = pd.concat(all_peaks.values(), ignore_index=True)
print(f"\nTotal peaks across all samples: {len(peaks_df)}")

## 4. Quality Control Metrics

In [ ]:
# Summary statistics
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Number of peaks per sample
peak_counts = peaks_df.groupby('sample').size()
peak_counts.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Number of Peaks per Sample')
axes[0].set_ylabel('Peak Count')
axes[0].tick_params(axis='x', rotation=45)

# Peak width distribution
peaks_df['width'] = peaks_df['end'] - peaks_df['start']
peaks_df.boxplot(column='width', by='sample', ax=axes[1])
axes[1].set_title('Peak Width Distribution')
axes[1].set_ylabel('Width (bp)')
axes[1].tick_params(axis='x', rotation=45)

# Signal value distribution
peaks_df.boxplot(column='signalValue', by='sample', ax=axes[2])
axes[2].set_title('Peak Signal Distribution')
axes[2].set_ylabel('Signal Value')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'qc_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. TSS Enrichment Analysis

TSS enrichment is a key quality metric for ATAC-seq. It measures the enrichment of reads around transcription start sites.

In [ ]:
def calculate_tss_enrichment(bam_file, tss_bed, window=2000, bin_size=50):
    """
    Calculate TSS enrichment from BAM file.
    
    Parameters:
    -----------
    bam_file : str/Path
        Path to BAM file
    tss_bed : str/Path
        Path to BED file with TSS coordinates
    window : int
        Window size around TSS (bp)
    bin_size : int
        Bin size for coverage calculation
    
    Returns:
    --------
    dict
        Dictionary with positions and coverage values
    """
    # Load TSS regions
    tss = BedTool(tss_bed)
    
    # Extend TSS regions to window size
    tss_extended = tss.slop(b=window, g=f"{GENOME_FASTA}.fai")
    
    # Calculate coverage
    bam = BedTool(str(bam_file))
    coverage = bam.coverage(tss_extended, d=True)
    
    # Parse coverage data
    positions = []
    depths = []
    
    for interval in coverage:
        pos = int(interval[3]) - window  # Relative position to TSS
        depth = int(interval[4])
        positions.append(pos)
        depths.append(depth)
    
    return {'position': positions, 'depth': depths}

# Calculate TSS enrichment for all samples
tss_enrichment = {}
for condition, samples in SAMPLES.items():
    for sample in samples:
        bam_file = BAM_DIR / f"{sample}.bam"
        print(f"Calculating TSS enrichment for {sample}...")
        tss_enrichment[sample] = calculate_tss_enrichment(bam_file, TSS_BED)

print("TSS enrichment calculation complete!")

In [ ]:
# Plot TSS enrichment
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot for hibernating samples
for sample in SAMPLES['hibernating']:
    data = pd.DataFrame(tss_enrichment[sample])
    data_binned = data.groupby(pd.cut(data['position'], bins=100))['depth'].mean()
    axes[0].plot(data_binned.index.categories.mid, data_binned.values, label=sample, alpha=0.7)

axes[0].set_xlabel('Distance from TSS (bp)')
axes[0].set_ylabel('Average Read Depth')
axes[0].set_title('TSS Enrichment - Hibernating')
axes[0].legend()
axes[0].axvline(x=0, color='black', linestyle='--', alpha=0.5)

# Plot for active samples
for sample in SAMPLES['active']:
    data = pd.DataFrame(tss_enrichment[sample])
    data_binned = data.groupby(pd.cut(data['position'], bins=100))['depth'].mean()
    axes[1].plot(data_binned.index.categories.mid, data_binned.values, label=sample, alpha=0.7)

axes[1].set_xlabel('Distance from TSS (bp)')
axes[1].set_ylabel('Average Read Depth')
axes[1].set_title('TSS Enrichment - Active')
axes[1].legend()
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'tss_enrichment.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate TSS enrichment score (ratio of reads at TSS vs flanking regions)
def calculate_tss_score(enrichment_data, tss_region=100, flank_region=(1000, 2000)):
    data = pd.DataFrame(enrichment_data)
    
    tss_reads = data[(data['position'] >= -tss_region) & 
                     (data['position'] <= tss_region)]['depth'].mean()
    
    flank_reads = data[((data['position'] >= -flank_region[1]) & (data['position'] <= -flank_region[0])) |
                       ((data['position'] >= flank_region[0]) & (data['position'] <= flank_region[1]))]['depth'].mean()
    
    return tss_reads / flank_reads if flank_reads > 0 else 0

print("\nTSS Enrichment Scores:")
for sample, data in tss_enrichment.items():
    score = calculate_tss_score(data)
    print(f"{sample}: {score:.2f}")

## 6. Peak Annotation

Annotate peaks to genomic features (promoter, exon, intron, intergenic, etc.)

In [ ]:
def annotate_peaks_to_genes(peaks_bed, gtf_file, promoter_distance=3000):
    """
    Annotate peaks to nearest genes and genomic features.
    
    Parameters:
    -----------
    peaks_bed : BedTool or str
        Peak regions
    gtf_file : str/Path
        GTF annotation file
    promoter_distance : int
        Distance from TSS to consider as promoter
    
    Returns:
    --------
    pd.DataFrame
        Annotated peaks with gene information
    """
    # Load GTF
    gtf = BedTool(str(gtf_file))
    
    # Extract genes
    genes = gtf.filter(lambda x: x[2] == 'gene').saveas()
    
    # Find nearest gene for each peak
    peaks = BedTool(peaks_bed) if isinstance(peaks_bed, str) else peaks_bed
    nearest = peaks.closest(genes, d=True)
    
    # Parse results
    annotations = []
    for feature in nearest:
        peak_info = {
            'peak_chrom': feature[0],
            'peak_start': int(feature[1]),
            'peak_end': int(feature[2]),
            'distance_to_gene': int(feature[-1])
        }
        
        # Parse gene information from GTF attributes
        if len(feature) > 10:
            attrs = feature[11]
            # Extract gene_id and gene_name
            import re
            gene_id_match = re.search(r'gene_id "([^"]+)"', attrs)
            gene_name_match = re.search(r'gene_name "([^"]+)"', attrs)
            
            peak_info['gene_id'] = gene_id_match.group(1) if gene_id_match else 'NA'
            peak_info['gene_name'] = gene_name_match.group(1) if gene_name_match else 'NA'
        
        # Classify genomic region
        if peak_info['distance_to_gene'] <= promoter_distance:
            peak_info['annotation'] = 'Promoter'
        elif peak_info['distance_to_gene'] == 0:
            peak_info['annotation'] = 'Genic'
        else:
            peak_info['annotation'] = 'Intergenic'
        
        annotations.append(peak_info)
    
    return pd.DataFrame(annotations)

# Annotate peaks for each condition
annotated_peaks = {}
for condition, samples in SAMPLES.items():
    for sample in samples:
        print(f"Annotating peaks for {sample}...")
        peak_file = PEAK_DIR / f"{sample}_peaks.narrowPeak"
        annotated = annotate_peaks_to_genes(str(peak_file), GTF_FILE)
        annotated['sample'] = sample
        annotated['condition'] = condition
        annotated_peaks[sample] = annotated

# Combine all annotations
all_annotated = pd.concat(annotated_peaks.values(), ignore_index=True)
print(f"\nAnnotated {len(all_annotated)} peaks total")

In [ ]:
# Visualize peak annotations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Genomic feature distribution
annotation_counts = all_annotated.groupby(['condition', 'annotation']).size().unstack(fill_value=0)
annotation_counts.plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_title('Peak Distribution Across Genomic Features')
axes[0].set_xlabel('Condition')
axes[0].set_ylabel('Number of Peaks')
axes[0].legend(title='Annotation')
axes[0].tick_params(axis='x', rotation=0)

# Distance to nearest gene
for condition in SAMPLES.keys():
    data = all_annotated[all_annotated['condition'] == condition]['distance_to_gene']
    # Log scale for better visualization
    data_log = np.log10(data[data > 0] + 1)
    axes[1].hist(data_log, bins=50, alpha=0.6, label=condition)

axes[1].set_title('Distance to Nearest Gene')
axes[1].set_xlabel('log10(Distance + 1)')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'peak_annotations.png', dpi=300, bbox_inches='tight')
plt.show()

# Summary table
print("\nPeak Annotation Summary:")
summary = all_annotated.groupby(['condition', 'annotation']).size().unstack(fill_value=0)
print(summary)

# Save annotated peaks
all_annotated.to_csv(OUTPUT_DIR / 'annotated_peaks.csv', index=False)
print(f"\nSaved annotated peaks to {OUTPUT_DIR / 'annotated_peaks.csv'}")

## 7. Differential Accessibility Analysis

Identify peaks that are differentially accessible between hibernating and active states.

In [ ]:
# Create consensus peak set
def create_consensus_peaks(peak_files, min_overlap=0.5):
    """
    Create a consensus peak set from multiple samples.
    
    Parameters:
    -----------
    peak_files : list
        List of peak file paths
    min_overlap : float
        Minimum fraction overlap to merge peaks
    
    Returns:
    --------
    BedTool
        Merged consensus peaks
    """
    # Concatenate all peaks
    all_peaks = BedTool(peak_files[0])
    for peak_file in peak_files[1:]:
        all_peaks = all_peaks.cat(BedTool(peak_file), postmerge=False)
    
    # Sort and merge overlapping peaks
    consensus = all_peaks.sort().merge()
    
    return consensus

# Get all peak files
all_peak_files = [str(PEAK_DIR / f"{sample}_peaks.narrowPeak") 
                  for samples in SAMPLES.values() for sample in samples]

consensus_peaks = create_consensus_peaks(all_peak_files)
consensus_peaks.saveas(OUTPUT_DIR / 'consensus_peaks.bed')
print(f"Created consensus peak set with {consensus_peaks.count()} peaks")

# Calculate read counts in consensus peaks for each sample
# This would typically use featureCounts or similar tool
# Placeholder for differential analysis
print("\nNote: For robust differential analysis, use DESeq2 or edgeR with read counts")

## 8. Motif Analysis

Identify enriched transcription factor binding motifs in ATAC-seq peaks.

In [ ]:
def extract_peak_sequences(peaks_bed, genome_fasta, output_fasta):
    """
    Extract DNA sequences for peak regions.
    
    Parameters:
    -----------
    peaks_bed : str/Path
        BED file with peak coordinates
    genome_fasta : str/Path
        Genome FASTA file
    output_fasta : str/Path
        Output FASTA file
    
    Returns:
    --------
    str
        Path to output FASTA file
    """
    peaks = BedTool(str(peaks_bed))
    sequences = peaks.sequence(fi=str(genome_fasta), fo=str(output_fasta))
    return str(output_fasta)

# Extract sequences for hibernating-specific peaks
# (You would first need to identify these from differential analysis)
hib_peaks_bed = OUTPUT_DIR / 'hibernating_specific_peaks.bed'
# For demonstration, use consensus peaks
consensus_peaks.saveas(hib_peaks_bed)

hib_sequences_fasta = OUTPUT_DIR / 'hibernating_peak_sequences.fa'
extract_peak_sequences(hib_peaks_bed, GENOME_FASTA, hib_sequences_fasta)
print(f"Extracted sequences to {hib_sequences_fasta}")

print("\nNext steps for motif analysis:")
print("1. Run HOMER: findMotifsGenome.pl peaks.bed genome output_dir -size 200")
print("2. Run MEME-ChIP: meme-chip -db motif_database.meme sequences.fa -oc output_dir")
print("3. Run AME for motif enrichment: ame --control control_sequences.fa sequences.fa motif_database.meme")

In [ ]:
# Example: Run HOMER motif finding (requires HOMER installation)
# Uncomment and modify paths as needed

# import subprocess

# homer_cmd = [
#     'findMotifsGenome.pl',
#     str(hib_peaks_bed),
#     str(GENOME_FASTA),
#     str(OUTPUT_DIR / 'homer_motifs'),
#     '-size', '200',
#     '-mask'
# ]

# print("Running HOMER motif analysis...")
# subprocess.run(homer_cmd, check=True)
# print("HOMER analysis complete!")

In [ ]:
# Parse and visualize HOMER results (if available)
def parse_homer_results(homer_dir):
    """
    Parse HOMER motif finding results.
    
    Parameters:
    -----------
    homer_dir : str/Path
        HOMER output directory
    
    Returns:
    --------
    pd.DataFrame
        Table of enriched motifs
    """
    motif_file = Path(homer_dir) / 'knownResults.txt'
    
    if not motif_file.exists():
        print(f"HOMER results not found at {motif_file}")
        return None
    
    motifs = pd.read_csv(motif_file, sep='\t')
    return motifs

# Example visualization
# homer_results = parse_homer_results(OUTPUT_DIR / 'homer_motifs')
# if homer_results is not None:
#     # Plot top motifs
#     top_motifs = homer_results.head(10)
#     
#     plt.figure(figsize=(10, 6))
#     plt.barh(range(len(top_motifs)), -np.log10(top_motifs['P-value']))
#     plt.yticks(range(len(top_motifs)), top_motifs['Motif Name'])
#     plt.xlabel('-log10(P-value)')
#     plt.title('Top Enriched Motifs in Hibernating Tenrec ATAC-seq Peaks')
#     plt.tight_layout()
#     plt.savefig(OUTPUT_DIR / 'top_motifs.png', dpi=300, bbox_inches='tight')
#     plt.show()

## 9. Integration with Gene Expression

Correlate chromatin accessibility with gene expression changes during hibernation.

In [ ]:
# Load RNA-seq data if available
# rna_seq_file = BASE_DIR / 'rnaseq' / 'differential_expression.csv'
# rna_seq = pd.read_csv(rna_seq_file)

# Merge with ATAC-seq data
# integrated = all_annotated.merge(rna_seq, 
#                                  left_on='gene_name', 
#                                  right_on='gene', 
#                                  how='inner')

# # Scatter plot: accessibility vs expression
# plt.figure(figsize=(8, 6))
# plt.scatter(integrated['log2FoldChange_ATAC'], 
#             integrated['log2FoldChange_RNA'],
#             alpha=0.5)
# plt.xlabel('ATAC-seq log2 Fold Change')
# plt.ylabel('RNA-seq log2 Fold Change')
# plt.title('Chromatin Accessibility vs Gene Expression')
# plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
# plt.axvline(x=0, color='black', linestyle='--', alpha=0.3)
# plt.tight_layout()
# plt.savefig(OUTPUT_DIR / 'atac_rna_correlation.png', dpi=300, bbox_inches='tight')
# plt.show()

print("Load RNA-seq data and uncomment the code above to perform integration analysis")

## 10. Functional Enrichment Analysis

Perform GO term and pathway enrichment for genes near differentially accessible peaks.

In [ ]:
# Extract genes near differential peaks
# differential_genes = all_annotated[
#     (all_annotated['annotation'] == 'Promoter') & 
#     (all_annotated['condition'] == 'hibernating')
# ]['gene_name'].unique()

# print(f"Found {len(differential_genes)} genes near hibernating-specific peaks")

# # Save gene list for enrichment analysis
# gene_list_file = OUTPUT_DIR / 'hibernating_genes.txt'
# pd.DataFrame(differential_genes, columns=['gene']).to_csv(
#     gene_list_file, index=False, header=False
# )

print("\nNext steps for functional enrichment:")
print("1. Use enrichR, DAVID, or Metascape for GO term enrichment")
print("2. Use GSEA for pathway analysis")
print("3. Check for enrichment of hibernation-related processes")
print("   - Metabolic suppression")
print("   - Cold adaptation")
print("   - Circadian rhythm")
print("   - Oxidative stress response")

## 11. Summary and Export

Generate summary statistics and export results.

In [ ]:
# Summary statistics
summary_stats = {
    'Total Peaks': len(all_annotated),
    'Consensus Peaks': consensus_peaks.count(),
    'Promoter Peaks': len(all_annotated[all_annotated['annotation'] == 'Promoter']),
    'Intergenic Peaks': len(all_annotated[all_annotated['annotation'] == 'Intergenic']),
    'Genes with Promoter Peaks': all_annotated[all_annotated['annotation'] == 'Promoter']['gene_name'].nunique(),
}

print("\n" + "="*50)
print("ANALYSIS SUMMARY")
print("="*50)
for key, value in summary_stats.items():
    print(f"{key}: {value}")

# Save summary
summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv(OUTPUT_DIR / 'analysis_summary.csv', index=False)

print(f"\n\nAll results saved to: {OUTPUT_DIR}")
print("\nGenerated files:")
for file in OUTPUT_DIR.glob('*'):
    print(f"  - {file.name}")

## 12. Additional Analyses (Optional)

### Footprinting Analysis
Identify transcription factor footprints within accessible regions.

In [ ]:
# Footprinting analysis using tools like:
# - TOBIAS
# - HINT-ATAC
# - PIQ

print("Footprinting analysis tools:")
print("1. TOBIAS: comprehensive TF footprinting pipeline")
print("2. HINT-ATAC: DNase/ATAC-seq footprinting")
print("3. Wellington: footprint detection algorithm")

### Chromatin State Analysis
Integrate with histone modification data if available.

In [ ]:
# ChromHMM or Segway for chromatin state segmentation
# Integration with ChIP-seq data for histone marks:
# - H3K4me3 (active promoters)
# - H3K27ac (active enhancers)
# - H3K4me1 (poised enhancers)
# - H3K27me3 (repressed regions)

print("Chromatin state analysis:")
print("1. Run ChromHMM on histone modification data")
print("2. Overlap ATAC-seq peaks with chromatin states")
print("3. Identify state transitions during hibernation")